In [1]:
from dotenv import load_dotenv

load_dotenv()

True

 ## Setup tools

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from typing import Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

## Create State

In [5]:
from langchain.agents import AgentState

class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

## Create Subagents

In [6]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

# Travel agent
travel_agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="""
    You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """
)

In [7]:
# Venue agent
venue_agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    """
)

In [8]:
# Playlist agent
playlist_agent = create_agent(
    model=model,
    tools=[query_playlist_db],
    system_prompt="""
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Try up to 3-4 different query approaches. If you still cannot get results after several attempts, 
    provide a brief explanation of what went wrong and suggest an alternative approach.
    """
)

## Verify Database Schema (Optional)

In [9]:
# Quick check to see available genres and track count
result = db.run("SELECT g.Name, COUNT(t.TrackId) as TrackCount FROM Genre g LEFT JOIN Track t ON g.GenreId = t.GenreId GROUP BY g.Name ORDER BY TrackCount DESC LIMIT 10")
print("Top 10 Genres by Track Count:")
print(result)

Top 10 Genres by Track Count:
[('Rock', 1297), ('Latin', 579), ('Metal', 374), ('Alternative & Punk', 332), ('Jazz', 130), ('TV Shows', 93), ('Blues', 81), ('Classical', 74), ('Drama', 64), ('R&B/Soul', 61)]


## Main Coordinator

In [10]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command


@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke(
        {"messages": [HumanMessage(content=f"Find flights from {origin} to {destination} on June 1st 2026")]}
    )
    return response["messages"][-1].content


@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response["messages"][-1].content


@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response["messages"][-1].content


@tool
def update_state(
    origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime
) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(
        update={
            "origin": origin,
            "destination": destination,
            "guest_count": guest_count,
            "genre": genre,
            "messages": [
                ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)
            ],
        }
    )

In [11]:
from langchain.agents import create_agent

coordinator = create_agent(
    model=model,
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)

## Test

In [12]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
    {
        "messages": [
            HumanMessage(
                content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre"
            )
        ],
    },
    config={"recursion_limit": 50},
)

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring


In [13]:
from pprint import pprint

pprint(response)

{'destination': 'Paris',
 'genre': 'jazz',
 'guest_count': '100',
 'messages': [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre", additional_kwargs={}, response_metadata={}, id='af290a31-592b-47a7-bc59-0c2b0ed9440f'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_state', 'arguments': '{"guest_count": "100", "origin": "London", "genre": "jazz", "destination": "Paris"}'}, '__gemini_function_call_thought_signatures__': {'76075e73-ee13-42d1-802e-e22147b5cbb0': 'CuQBAXLI2nzjFBUaTG96cfW/6O332n5PAnxeaB44UDRg7hQUgMYBOaZRnfrPVI1cFy+fuOO0cgT68OU1eX85Gth9whrm5If5GSb7SudWJcEcxYMHSAm6EK8RQkH7MubUNrFR5okwQzR4ftw9jBASRI4a2a69iAcWIMe7LItpHl0KKnWvZ1zQ5qC3P5ryvj7RjiUHwbdD4O0T6eNnNU1egIMGFa6YezHwFvSm02ADEAcLeOsZtstqj3MdLUzbvCPopqouuewZ5+ak7TS8yzfyGnEmeag8tixn7vMwHFCQj9IURneT8jUv'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}

In [15]:
pprint(response["messages"][-1].content)

[{'extras': {'signature': 'CpsFAXLI2nzjUqcgMQugq2lEFObDECjeRJkM0b/De3VSvUayH3RXS4XBxA7UyEAyOQ7xz/vZ+5KquDJ2b8EvvS5G2JvnAZ1DRcEiD5NHR+9v4oltbUd5a9CW48KlJQnGYReEqn4lNYRO+QvXxThwT1i2Pb9iWtPVT0LquzZIX7fWYFn79ZNeH8HaF0G2oF+HfMMFb1rOsivauLguEQG5kmwfYJrDeEF51CFFsdcUGan3PyhaCKGgwHAhbf4tKZIqqE/bsC17md+HXuq7/rk7mNuajWt6bOUBdMGzO8gYvRtYJznhA3ZYU+KJQQ8OmBg/6kX2WQqx6+nQIa1h6y3O8bi/ZOvJLjhGbV5yK9giWG2SbRYx85Y7moeDsHQTNxNC/VrGnpvLRMib5dT8B4swc2oxuds3CAasjRe2MT+BMx72IonODGmJpP//pC7YwgAGwD70YLjnVR4Q+KHy6sf6vINYtnnEvyj3sSa7sO077UbIzZmvho3/3pla9PnWlFL6QOLY8kH/qTP9KR/ZYNWh98/q8pE3b07sPCtWD4syB7+Y+kay9r04DTrNcARo4GdhJDw01/VDiPH6z8S6jVILVoTPy4skF6akZF1m5GPsT5XqHG5nfFwKNblaAAp0VGXknd0ZpI7uWYzmHQQVPnOSU0NNK2y713qTWdgPh4A1trRxwWhjRWRczVIrUGDYigwIK+uuNbvVwr1HPWqMGbAoMlcVw9SlwaS3/BEKJzadS4EXZ5Fp8s/Mrow+4/sJCOHnGDzb3IPwpg2z9EMcIHazAw/DD8c6nQin9zqCdtk6W/xXIA/OEDsfN4arumD8DJmb4/AO5/WJQsg467bep58nIOGMPqFJOI/bDcWnxjZkfDrhzXsm8z6ToxpDUb885Q=='},
  'text': "Here's your wedding plan for a beautiful jazz-themed wedding i

Trace: https://smith.langchain.com/public/6d255b1a-7812-4147-a901-f421b68c527b/r